In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyPDF2

In [ ]:
import pandas as pd

In [ ]:
!pip install transformers

In [ ]:
import PyPDF2
import re
from transformers import GPT2TokenizerFast
import pandas as pd

# Load PDF file
pdf_file = open('/content/drive/MyDrive/Datasets/DeepFake_detection_algorithm_based_on_improved_vis (1).pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Extract text from PDF
text = ''
for page in range(len(pdf_reader.pages)):
    text += pdf_reader.pages[page].extract_text()

# Split text into sections by headings
headings = re.findall(r'\n(.+?)\n', text)
sections = re.split(r'\n.+?\n', text)

# Remove empty strings from sections
sections = [s.strip() for s in sections if s.strip()]

# Tokenize sections
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokens = [tokenizer.encode(s) for s in sections]

# Create dataframe
df = pd.DataFrame({
    'title': ['Research Paper'] * len(headings),
    'heading': headings,
    'content': sections[0:433],
    'token': tokens[0:433]
})


In [ ]:
df.head()

,title,heading,content,token
0,Research Paper,https://doi.org/10.1007/s10489-022-03867-9,AppliedIntelligence,"[4677, 18511, 5317, 3480]"
1,Research Paper,transformer,DeepFakedetectionalgorithmbasedonimprovedvision,"[29744, 37, 4335, 316, 478, 1538, 7727, 2022, ..."
2,Research Paper,Accepted:7June2022,Young-JinHeo1·Woon-HaYeo1·Byung-GyuKim1,"[20917, 12, 41, 259, 1544, 78, 16, 9129, 54, 2..."
3,Research Paper,Abstract,"©TheAuthor(s),underexclusivelicencetoSpringerS...","[16224, 464, 13838, 7, 82, 828, 917, 567, 87, ..."
4,Research Paper,networks or auto encoders that anyone can util...,A DeepFake is a manipulated video made with ge...,"[32, 10766, 49233, 318, 257, 25036, 2008, 925,..."


In [ ]:
len(df)

433

In [ ]:
def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

In [ ]:
def reduce_long(
    long_text: str, long_text_tokens: bool = False, max_len: int = 590
) -> str:
    
    if not long_text_tokens:
        long_text_tokens = count_tokens(long_text)
    if long_text_tokens > max_len:
        sentences = sent_tokenize(long_text.replace("\n", " "))
        ntokens = 0
        for i, sentence in enumerate(sentences):
            ntokens += 1 + count_tokens(sentence)
            if ntokens > max_len:
                return ". ".join(sentences[:i][:-1]) + "."

    return long_text

In [ ]:
df['heading'].head(20)

0            https://doi.org/10.1007/s10489-022-03867-9
1                                           transformer
2                                    Accepted:7June2022
3                                              Abstract
4     networks or auto encoders that anyone can util...
5     overfitting and cannot consider the relation b...
6     features. We combine vector-concatenated CNN f...
7     By adding this distillation, the proposed mode...
8     the proposed model correctly predicts 2,313 as...
9     proposed model achieves a high performance of ...
10                                        1Introduction
11    bilities, deep learning models have been emplo...
12    network (CNN) models are used in the fields of...
13    have emerged as state-of-the-art (SOTA) method...
14    ial network (GAN) face generation models perfo...
15    GAN [ 7]. Deepfake videos can easily be made u...
16                           /envelopebackByung-Gyu Kim
17                                        Young-

In [ ]:
max_len = 500
for c in sections:
   ncontent_ntokens = [
        count_tokens(reduce_long(c,max_len))
        + 3
        + count_tokens(" ".join(h.split(" ")[1:-1]))
        - (1 if len(c) == 0 else 0)
        for h, c in zip(headings, sections)
    ]


In [ ]:
max_len = 500
df = pd.DataFrame({
    'title': ['Research Paper'] * 431,
    'heading': headings[0:431],
    'content': sections[0:431],
    'token': ncontent_ntokens[0:431]
})


In [ ]:
len(df)

431

In [ ]:
df.head()

,title,heading,content,token
0,Research Paper,https://doi.org/10.1007/s10489-022-03867-9,AppliedIntelligence,7
1,Research Paper,transformer,DeepFakedetectionalgorithmbasedonimprovedvision,16
2,Research Paper,Accepted:7June2022,Young-JinHeo1·Woon-HaYeo1·Byung-GyuKim1,26
3,Research Paper,Abstract,"©TheAuthor(s),underexclusivelicencetoSpringerS...",33
4,Research Paper,networks or auto encoders that anyone can util...,A DeepFake is a manipulated video made with ge...,46


In [ ]:
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

431 rows in the data.


content  \
title          heading                                                                                                 
Research Paper Woon-Ha Yeo received the                            Content courtesy of Springer Nature, terms of ...   
               Member for the International Journal of Soft Co...  journals, including IEEE, ACM, Elsevier, Sprin...   
               unsuitable features, they affect the specific a...  each element to find the deeply related parts....   
               Section 3.2.4 .                                     illustrate that how the input consists in Sect...   
               Her research interests include                                                Co., Republic of Korea.   

                                                                   token  
title          heading                                                    
Research Paper Woon-Ha Yeo received the                               29  
               Member for the International Journal of Soft Co...     55  
               unsuitable features, they affect the specific a...     33  
               Section 3.2.4 .                                        34  
               Her research interests include                         11

In [ ]:
!pip install openai

In [ ]:
!pip install tiktoken

In [ ]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [ ]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict:
   
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [ ]:
def load_embeddings(fname: str) -> dict:
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [ ]:
from getpass import getpass

In [ ]:
openai.api_key = getpass()

··········


In [ ]:
from openai.embeddings_utils import get_embedding

In [ ]:
def compute_doc_embeddings(df: pd.DataFrame) -> dict:
   
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [ ]:
df.size

862

In [ ]:
import time
import openai

# Define a function that adds a delay to a Completion API call
def delayed_completion(df, delay_in_seconds: float = 1):

    # Sleep for the delay
    time.sleep(delay_in_seconds)

    # Call the Completion API and return the result
    return compute_doc_embeddings(df)





In [ ]:
rate_limit_per_minute = 20
delay = 60.0 / rate_limit_per_minute

document_embeddings = delayed_completion(df[:1], delay_in_seconds=delay)


In [ ]:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

In [ ]:
def vector_similarity(x: list, y: list) -> float:
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict) -> list:
    
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [ ]:
order_document_sections_by_query_similarity("What is deepfake according to this paper?", document_embeddings)[:5]

In [ ]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [ ]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
   
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [ ]:
prompt = construct_prompt(
    "What is the technology behind DeepFake?",
    document_embeddings,
    df
)

print("===\n", prompt)

In [ ]:
COMPLETIONS_API_PARAMS = {
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [ ]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [ ]:
query = "What is the difference between Generator and Discriminator?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")